In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import tensorflow_datasets as tfds

In [56]:
ds, ds_info = tfds.load('food101', shuffle_files=True, as_supervised=True, with_info=True) # type: ignore

In [57]:
train_ds, valid_ds = ds['train'], ds['validation'] # type: ignore


In [58]:
MAX_SIZE = 128
HOTDOG_CLASS = 55

train_ds = train_ds.map(
    lambda image, label: (
        tf.cast(tf.image.resize(image, [MAX_SIZE, MAX_SIZE]),
                dtype=tf.int32),
        tf.cast(label == HOTDOG_CLASS, dtype=tf.int32)
    )
)

valid_ds = valid_ds.map(
    lambda image, label: (
        tf.cast(tf.image.resize(image, [MAX_SIZE, MAX_SIZE]),
                dtype=tf.int32),
        tf.cast(label == HOTDOG_CLASS, dtype=tf.int32)
    )
)

In [59]:
train_hd_size, valid_hd_size = 750, 250
train_hotdogs = train_ds.filter(lambda _image, label: label == 1).repeat(3)
train_nothotdogs = train_ds.filter(lambda _image, label: label == 0)

In [60]:
valid_hotdogs = valid_ds.filter(lambda _image, label: label == 1).repeat(3)
valid_nothotdogs = valid_ds.filter(lambda _image, label: label == 0)

In [61]:
batch_size = 16

train_ds = tf.data.Dataset.sample_from_datasets([train_hotdogs, train_nothotdogs], [0.5, 0.5], stop_on_empty_dataset=True)

train_ds = train_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

valid_ds = tf.data.Dataset.sample_from_datasets([valid_hotdogs, valid_nothotdogs], [0.5, 0.5], stop_on_empty_dataset=True)

valid_ds = valid_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

NN Implementation

In [66]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2)
])

In [67]:
random.seed(0)
model = models.Sequential()

model.add(layers.Rescaling(1./255))
model.add(data_augmentation)
model.add(layers.Conv2D(64, (3, 3), activation='relu',
                        input_shape=(MAX_SIZE, MAX_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3, 3), activation='relu',
          kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(32, (3, 3), activation='relu',
          kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1))

In [68]:
lr = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [69]:
epochs = 5
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=epochs,
    verbose=1
)

Epoch 1/5
275/275 [==============================] - 733s 3s/step - loss: 1.4372 - accuracy: 0.4892 - val_loss: 1.1708 - val_accuracy: 0.5120
Epoch 2/5
275/275 [==============================] - 174s 630ms/step - loss: 1.0187 - accuracy: 0.4917 - val_loss: 0.9077 - val_accuracy: 0.5205
Epoch 3/5
275/275 [==============================] - 172s 625ms/step - loss: 0.8438 - accuracy: 0.5163 - val_loss: 0.7952 - val_accuracy: 0.5986
Epoch 4/5
275/275 [==============================] - 161s 587ms/step - loss: 0.7684 - accuracy: 0.5277 - val_loss: 0.7456 - val_accuracy: 0.6025
Epoch 5/5
275/275 [==============================] - 176s 639ms/step - loss: 0.7329 - accuracy: 0.5365 - val_loss: 0.7195 - val_accuracy: 0.6129
